# Analsis de Datos con LangChain

In [1]:
!pip install langchain langchain_openai langchain_experimental pandas numpy tabulate -q

In [12]:
!pip install openpyxl

In [2]:
from langchain_openai import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain.schema.output_parser import StrOutputParser
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
import os


In [3]:
df = pd.read_csv("CSVs/goalscorers.csv")
df2 = pd.read_csv("CSVs/results.csv")
df3 = pd.read_csv("CSVs/shootouts.csv")

In [4]:
df.head()

,date,home_team,away_team,team,scorer,minute,own_goal,penalty
0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,False,False
1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,False,False
2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,False,False
3,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,75.0,False,False
4,1916-07-06,Argentina,Chile,Argentina,Alberto Ohaco,2.0,False,False


In [5]:
df2.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [6]:
df3.head()

,date,home_team,away_team,winner,first_shooter
0,1967-08-22,India,Taiwan,Taiwan,NaN
1,1971-11-14,South Korea,Vietnam Republic,South Korea,NaN
2,1972-05-07,South Korea,Iraq,Iraq,NaN
3,1972-05-17,Thailand,South Korea,South Korea,NaN
4,1972-05-19,Thailand,Cambodia,Thailand,NaN


In [7]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

In [9]:
load_dotenv(find_dotenv())

True

In [10]:
openai.api_key = os.environ['OPENAI_API_KEY']

In [11]:
model = "gpt-3.5-turbo-16k"

In [13]:
llm_model = ChatOpenAI(model = model, temperature=0,verbose=True, streaming=True)

In [14]:
df_parser = PandasDataFrameOutputParser(dataframe=df)

In [15]:
print(df_parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [19]:
prompt = PromptTemplate(
    template = "Responde al usuario su consulta.\n{format_instructions}\n{query}\n",
    input_variables = ["query"],
    partial_variables = {"format_instructions" : df_parser.get_format_instructions()},
)

In [20]:
chain = prompt | llm_model | df_parser

In [21]:
chain.invoke(
    {"query": "dame la primera fila."
    })

{'0': date              1916-07-02
 home_team              Chile
 away_team            Uruguay
 team                 Uruguay
 scorer       José Piendibene
 minute                  44.0
 own_goal               False
 penalty                False
 Name: 0, dtype: object}

In [24]:
chain.invoke(
    {"Dime el nombre de la segunda columna y los tipos de valores en ella"
    })

{'home_team': 0            Chile
 1            Chile
 2            Chile
 3            Chile
 4        Argentina
            ...    
 44357       Canada
 44358    Argentina
 44359        Spain
 44360        Spain
 44361        Spain
 Name: home_team, Length: 44362, dtype: object}

In [26]:
chain.invoke(
    {"Dime el valor mas repetido de la segunda columna"
    })

OutputParserException: Invalid operation: Dime el valor mas repetido de la segunda columna.. Please check the format instructions.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE